In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import datetime
import matplotlib.pyplot as plt
from fpdf import FPDF
from concurrent.futures import ThreadPoolExecutor

c:\Users\slowa\AppData\Local\Programs\Python\Python311\Lib\site-packages\fpdf\__init__.py:39: UserWarning: You have both PyFPDF & fpdf2 installed. Both packages cannot be installed at the same time as they share the same module namespace. To only keep fpdf2, run: pip uninstall --yes pypdf && pip install --upgrade fpdf2
  warnings.warn(


In [2]:
if os.path.exists("szukana_wartosc.txt"):
    print("Plik istnieje!")
    wyszukiwanie=True
    with open("szukana_wartosc.txt", 'r') as plik:
        # Odczyt zawartości pliku
        zawartosc = plik.read().split(';')        
        tryb_szukania=zawartosc[0]
        szukana_wartość=zawartosc[1]       
    os.remove("szukana_wartosc.txt")

else:
    print("Plik nie istnieje.")
    wyszukiwanie=False
    # Odczytaj wartość zmiennej z pliku
    with open('przekazana_zmienna.txt', 'r') as f:
        tresc=f.read()
        przekazana_zmienna = int(tresc.splitlines()[-1][-2:]) if tresc.splitlines()[-1][-2:].isdigit() else int(tresc.splitlines()[-1][-1])
    os.remove('przekazana_zmienna.txt')

Plik nie istnieje.


FileNotFoundError: [Errno 2] No such file or directory: 'przekazana_zmienna.txt'

In [3]:
kategorie=["data","miejsce","wyksztalcenie","szkola","tytul","zawod","lista","okreg","glosy","funkcja,""slubowanie","wygasniecie"]
poslowie={}

#dodawanie posła do słownika
def dodaj_posla(imie,lista2):
    posel = dict.fromkeys(kategorie)
    poslowie.update({imie: posel})
    for klucz, wartosc in zip(posel,lista2):
        posel[klucz]=wartosc

In [13]:
def jedna_szkola(x):
    lista0=set()
    x=x.replace("<p class=\"right\">", "").replace("</p>", "").split('<br/>')
    for y in x:
        y=y.split('(')[0].split(',')[0]
        lista0.add(y) 
    return(lista0)


def process_posel(data):
    soup2, imie, current_html, kadencja = data
    dane = soup2.select('body table tr td li p')
    
    for q in range(1, len(dane),2):
        tytul=""
        wygasniecie=""
        funkcja=""

        if (dane[q].text.strip()=="Data i miejce urodzenia:"):
            try:
                data=datetime.datetime.strptime(dane[q+1].text.split(",")[0].strip(),"%d-%m-%Y").date()
                if wyszukiwanie:
                    if tryb_szukania=="Rok urodzenia":
                        if str(data.year)==szukana_wartość:
                            znalezione.append([imie.text,current_html,kadencja])
            except ValueError:
                # Obsługa błędu indeksu poza zakresem, dla posłów którzy nie mają wprowadzonego miejsca urodzenia
                print("Poseł "+imie.text+" nie posiada wprowadzonej daty urodzenia")
                print(dane[q+1].text.split(",")[0].strip())
            try:                
                miejsce=dane[q+1].text.split(",")[1].strip()
                miejsca_urodzenia.add(dane[q+1].text.split(",")[1].strip())
                if wyszukiwanie:
                    if tryb_szukania=="Miejsce urodzenia":
                        if miejsce.lower()==szukana_wartość.lower():
                            znalezione.append([imie.text,current_html,kadencja])
            except IndexError:
                # Obsługa błędu indeksu poza zakresem, dla posłów którzy nie mają wprowadzonego miejsca urodzenia
                print("Błąd: Indeks poza zakresem")
                print(dane[q+1].text)
        elif (dane[q].text.strip()=="Wykształcenie:"):
            wyksztalcenie=dane[q+1].text.strip()
            poziom_wyksztalcenia.add(dane[q+1].text.strip())
            if wyszukiwanie:
                if tryb_szukania=="Poziom wykształcenia":
                    if wyksztalcenie.lower()==szukana_wartość.lower():
                        znalezione.append([imie.text,current_html,kadencja])
        elif (dane[q].text.strip()=="Ukończona szkoła:"):
            szkola=jedna_szkola(str(dane[q+1]))
            for x in szkola:
                szkoly.add(x)
                if wyszukiwanie:
                    if tryb_szukania=="Ukończona szkoła":
                        if szukana_wartość.lower() in x.lower():
                            znalezione.append([imie.text,current_html,kadencja])
        elif (dane[q].text.strip()=="Zawód:" or dane[q].text.strip()=="Zawód/stanowisko:"):
            zawod=dane[q+1].text.strip()
            zawody.add(dane[q+1].text.strip())
            if wyszukiwanie:
                if tryb_szukania=="Zawód":
                        if szukana_wartość.lower() in zawod.lower():
                            znalezione.append([imie.text,current_html,kadencja])
        elif (dane[q].text.strip()=="Lista:"):
            lista=dane[q+1].text.strip()
            listy.add(dane[q+1].text.strip())
        elif (dane[q].text.strip()=="Okręg wyborczy:"):
            okreg=dane[q+1].text.strip()
            okregi.add(dane[q+1].text.strip())    
        elif (dane[q].text.strip()=="Liczba głosów:"):
            glosy=dane[q+1].text.strip()
        elif (dane[q].text.strip()=="Ślubowanie:"):
            slubowanie=dane[q+1].text.strip()
        elif (dane[q].text.strip()=="Funkcja w Sejmie:"):
            funkcja=dane[q+1].text.strip()
            funkcje.add(dane[q+1].text.strip())
        elif (dane[q].text.strip()=="Tytuł/stopień naukowy:"):
            tytul=dane[q+1].text.strip()
            tytuly.add(dane[q+1].text.strip())
        elif (dane[q].text.strip()=="Wygaśnięcie\xa0mandatu:"):
            wygasniecie=dane[q+1].text.strip()
        else:
            nieznane.add(dane[q].text)
    lista2=[data,miejsce,wyksztalcenie,szkola,tytul,zawod,lista,okreg,glosy,funkcja,slubowanie,wygasniecie]
    dodaj_posla(imie.text,lista2)



if wyszukiwanie:
    znalezione=[]
    przekazana_zmienna=licznik = 9 if zawartosc[2] == "e" else 1 #ostatni znak przy wyborcze "wszystkie kadencje"
else:
    print("tutaj")
    licznik=1

executor = ThreadPoolExecutor(max_workers=10) # maksymalnie 10 wątków

while licznik!=0: #pętla działająca przy wyborcze wszystkich kadencji
    start_url = f"https://orka.sejm.gov.pl/ArchAll2.nsf/kad{przekazana_zmienna}RP"
    licznik-=1
    if licznik!=0:
        przekazana_zmienna=licznik
    kadencja=przekazana_zmienna
    
    download_html = requests.get(start_url)
    soup = BeautifulSoup(download_html.text)
    with open('downloaded.html', 'w', encoding="utf-8") as file:
        file.write(soup.prettify()) 

    full_table = soup.select('body table tr td a')

    nieznane=set()
    poziom_wyksztalcenia=set()
    miejsca_urodzenia=set()
    zawody=set()
    listy=set()
    okregi=set()
    funkcje=set()
    tytuly=set()
    szkoly=set()
    zawod=""

    for i in range(0, len(full_table), 2):
        current_html=("https://orka.sejm.gov.pl"+full_table[i].get("href"))
        download2_html=requests.get(current_html)
        soup2=BeautifulSoup(download2_html.text)
        imie = (soup2.find(class_= 'posel'))
        
        # Przekazujemy dane do funkcji process_posel, uruchamianej w osobnych wątkach
        executor.submit(process_posel, (soup2, imie, current_html, kadencja))

executor.shutdown(wait=True) # czekamy na zakończenie wszystkich wątków
print("Wszystkie wątki zostały zakończone.")

# Tu znajduje się reszta Twojego kodu...

tutaj
Poseł Wojciech Borzuchowski nie posiada wprowadzonej daty urodzenia
-1--1--1
Poseł Józef Krzyworzeka nie posiada wprowadzonej daty urodzenia
-1--1--1
Wszystkie wątki zostały zakończone.


In [58]:

def tabela_danych(dane,tytul_poslowie,naglowek):          
          slownik = {element: 0 for element in dane}
          if bool(slownik):
               for x,y in poslowie.items():
                    if type(y[tytul_poslowie])==set:
                         for z in y[tytul_poslowie]:
                              slownik[z]=slownik[z]+1
                    else:
                         slownik[y[tytul_poslowie]]=slownik[y[tytul_poslowie]]+1

          return( pd.DataFrame(list(slownik.items()), columns =[naglowek, 'Liczba posłów']))

if not wyszukiwanie:
     df=tabela_danych(miejsca_urodzenia,"miejsce","Miasta")
     df2=tabela_danych(poziom_wyksztalcenia,"wyksztalcenie","Wykształcenie")
     df3=tabela_danych(zawody,"zawod","Zawód")
     df4=tabela_danych(listy,"lista","Lista")
     df5=tabela_danych(szkoly,"szkola","Szkoła")
else:
     df=pd.DataFrame(znalezione, columns =['Imię i nazwisko', 'Link',"Kadencja"])


True
<class 'dict'>


In [60]:
if not wyszukiwanie:
     miejsca_urodzenia=sorted(miejsca_urodzenia)

     slownik = {element: 0 for element in miejsca_urodzenia}

     for x,y in poslowie.items():
          slownik[y["miejsce"]]=slownik[y["miejsce"]]+1

     df = pd.DataFrame(list(slownik.items()), columns =['Miasta', 'Liczba posłów'])

     #WYKSZTAŁCENIE
     slownik2 = {element: 0 for element in poziom_wyksztalcenia}

     for x,y in poslowie.items():
          slownik2[y["wyksztalcenie"]]=slownik2[y["wyksztalcenie"]]+1

     df2 = pd.DataFrame(list(slownik2.items()), columns =['Wyksztalcenie', 'Liczba posłów'])

     #ZAWÓD
     if bool(zawody):
          slownik3 = {element: 0 for element in zawody}

          for x,y in poslowie.items():
               slownik3[y["zawod"]]=slownik3[y["zawod"]]+1

          df3 = pd.DataFrame(list(slownik3.items()), columns =['Zawod', 'Liczba posłów'])

     #LISTY
     slownik4 = {element: 0 for element in listy}

     for x,y in poslowie.items():
          slownik4[y["lista"]]=slownik4[y["lista"]]+1

     df4 = pd.DataFrame(list(slownik4.items()), columns =['Lista', 'Liczba posłów'])

In [61]:
if not wyszukiwanie:
    def sortowanie(dane, liczba):
        return dane.sort_values(by='Liczba posłów', ascending=False).head(liczba)
    top_10_miast=sortowanie(df,10)
    df_sorted2=sortowanie(df2,len(df2))
    top_10_zawodow=sortowanie(df3,10)
    df_sorted4=sortowanie(df4,len(df4))
    top10_uczelni=sortowanie(df5,10)
else:
    nazwiska = set([tablica[0] for tablica in znalezione])

In [62]:
      
def nowypdf(x):
    pdf = FPDF()
    pdf.add_font('lato', 'B', 'Lato-Bold.ttf')  # Dodanie niestandardowej czcionki
    pdf.add_font('Lato', '', 'Lato-Regular.ttf', uni=True)  # Dodanie niestandardowej czcionki
    pdf.add_page()
    pdf.set_font('Lato', '', 12)  # Ustawienie niestandardowej czcionki
    pdf.set_text_color(255, 165, 0)
    if x==1:
        pdf.set_font('Lato', 'B', 20)        
        pdf.cell(0, 10, ("Analiza statystyczna posłów "+ str(przekazana_zmienna)+" kadencji sejmu").upper(), align='C', ln=True)
        pdf.set_font('Lato', '', 12)  # Ustawienie niestandardowej czcionki
        pdf.set_text_color(0,0,0)
        
        pdf.write(4,"\nŁącznie w wybranej kadencji sejmu pojawiło się " + str(len(poslowie)) + " posłów.\n")
        pdf.write(4,"\nPoniżej przedstawiono wykresy pokazujące dane statystyczne.\n")
    else:
        pdf.set_font('Lato', 'B', 20)        
        pdf.cell(0, 10, ("Wyszukiwanie wyrażenia: "+ str(szukana_wartość)).upper(), align='C', ln=True)
        pdf.cell(0, 10, ("dla pola: "+str(tryb_szukania)).upper(), align='C', ln=True)
        pdf.set_font('Lato', '', 12)  # Ustawienie niestandardowej czcionki
        pdf.set_text_color(0,0,0)
        
        pdf.write(4,"\nŁącznie znaleziono " + str(len(nazwiska)) + " posłów spełniających wyszukiwane kryteria.\n\n")
    return pdf



def generowanie_wykresu(dane, kolumna1, tytuł, pdf, numer):
    plt.rcParams['font.family'] = 'Lato'

    # Tworzenie wykresu słupkowego z dokładnymi wartościami
    plt.figure(figsize=(10, 6))
    bars = plt.bar(dane[kolumna1], dane['Liczba posłów'], color='orange', edgecolor='green')

    # Dodanie dokładnych wartości przy każdym słupek
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')

    plt.xlabel(kolumna1)
    plt.ylabel('Liczba posłów')
    plt.title(tytuł, fontsize=20, color="orange")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(f'wykres{numer}.png')  # Zmieniona nazwa pliku

    # Dodaj wykres do pliku PDF
    page_width = pdf.w - 2 * pdf.l_margin
    pdf.image(f'wykres{numer}.png',w=page_width)
    # pdf.add_page()  # Przejdź do nowej strony po dodaniu wykresu
    return(pdf)
if not wyszukiwanie:
    pdf = nowypdf(1)
    pdf=generowanie_wykresu(top_10_miast, "Miasta", 'Top 10 miast z największą liczbą posłów', pdf, 1)
    pdf=generowanie_wykresu(df_sorted2, "Wyksztalcenie", 'Liczba posłów w zależności od wykształcenia', pdf, 2)
    if bool(zawody):
        pdf=generowanie_wykresu(top_10_zawodow, "Zawod", 'Top 10 zawodów wykonywanych przez posłów', pdf, 3)
    pdf=generowanie_wykresu(df_sorted4, "Lista", 'Liczba posłów w zależności od listy wybrania', pdf, 4)
    pdf=generowanie_wykresu(top10_uczelni, "Szkoła", 'Liczba posłów w zależności od ukończonej uczelni', pdf, 5)
    # Zapisz do pliku PDF
    pdf.output('plik.pdf')
    %run zapis.ipynb

else:
    pdf = nowypdf(0)
    tytul=['Poseł','Link','Kadencja']
    with pdf.table(text_align="CENTER", col_widths=(80, 80, 40)) as table:
        row = table.row()  # Dodaj pierwszy wiersz
        for data_row in tytul:
            pdf.set_fill_color(255, 165, 0)
            row.cell(data_row)
            pdf.set_fill_color(255, 255, 255)
        for _, data_row in df.iterrows():
            row = table.row()
            for i, datum in enumerate(data_row.tolist()):
                if i == 1:
                    pdf.set_text_color(0, 0, 255)  # Ustaw kolor tekstu na niebieski
                    row.cell(text="LINK",link=datum)
                    pdf.set_text_color(0, 0, 0)  # Ustaw kolor tekstu na niebieski
                else:
                    row.cell(str(datum))


    pdf.output('plik.pdf')
    %run zapis.ipynb

C:\Users\slowa\AppData\Local\Temp\ipykernel_27100\1444448067.py:4: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  pdf.add_font('Lato', '', 'Lato-Regular.ttf', uni=True)  # Dodanie niestandardowej czcionki
C:\Users\slowa\AppData\Local\Temp\ipykernel_27100\1444448067.py:18: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, ("Wyszukiwanie wyrażenia: "+ str(szukana_wartość)).upper(), align='C', ln=True)
C:\Users\slowa\AppData\Local\Temp\ipykernel_27100\1444448067.py:19: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, ("dla pola: "+str(tryb_szukania)).upper(), align='C', ln=True)
